In [ ]:
import os
from dotenv import load_dotenv
from huggingface_hub import login
from datasets import load_dataset
from tqdm.auto import tqdm

load_dotenv(override=True)

: 

In [3]:
hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

Token has not been saved to git credential helper.
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.


In [8]:
dataset = load_dataset("semeru/code-text-javascript", split="train")

In [13]:
print(f"Number of Datas: {len(dataset):,}")
print(dataset)

Number of Datas: 58,025
Dataset({
    features: ['repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url', 'partition'],
    num_rows: 58025
})


In [22]:

max_len = 0
max_item = None

for datapoint in tqdm(dataset):
    try:
        cur_len = len(datapoint["original_string"])
        if cur_len > max_len:
            max_item = datapoint
            max_len = cur_len
    except ValueError:
        pass

print(f"The most expensive item is {max_item['func_name']} and it costs {max_len:,.2f}")

100%|██████████| 58025/58025 [00:08<00:00, 6525.73it/s]

The most expensive item is emitFiles and it costs 372,627.00
